# IHLT Final Project: Semantinc Textual Similarity
Jordi Armengol - Joan LLop

## Data collection
We start by downloading the SemEval 2012 dataset.

In [7]:
!mkdir -p data
!wget https://gebakx.github.io/ihlt/sts/resources/train.tgz --directory-prefix=data
!wget https://gebakx.github.io/ihlt/sts/resources/test-gold.tgz --directory-prefix=data
%cd data
!tar zxvf train.tgz
!tar zxvf test-gold.tgz
%cd ..

--2019-12-06 11:32:53--  https://gebakx.github.io/ihlt/sts/resources/train.tgz
Resolving gebakx.github.io (gebakx.github.io)... 185.199.109.153, 185.199.108.153, 185.199.110.153, ...
Connecting to gebakx.github.io (gebakx.github.io)|185.199.109.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 125822 (123K) [application/octet-stream]
Saving to: ‘data/train.tgz’

train.tgz           100%[===================>] 122,87K  --.-KB/s    in 0,04s   

2019-12-06 11:32:54 (3,16 MB/s) - ‘data/train.tgz’ saved [125822/125822]

--2019-12-06 11:32:54--  https://gebakx.github.io/ihlt/sts/resources/test-gold.tgz
Resolving gebakx.github.io (gebakx.github.io)... 185.199.111.153, 185.199.108.153, 185.199.110.153, ...
Connecting to gebakx.github.io (gebakx.github.io)|185.199.111.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 118094 (115K) [application/octet-stream]
Saving to: ‘data/test-gold.tgz’

test-gold.tgz       100%[===================>] 115,

## Corpus assembly

In [8]:
import os
train_files = ['MSRpar', 'MSRvid', 'SMTeuroparl']
train_data = []
train_labels = []
for file in train_files:
    with open(os.path.join('data', 'train', 'STS.input.' + file + '.txt'), 'r') as f:
        train_data += [sent.split('\t') for sent in f.readlines()]
    with open(os.path.join('data', 'train', 'STS.gs.' + file + '.txt'), 'r') as f:
        train_labels += [float(num) for num in f.readlines()]
test_files = ['MSRpar', 'MSRvid', 'SMTeuroparl', 'surprise.OnWN', 'surprise.SMTnews']
test_data = []
test_labels = []
for file in test_files:
    with open(os.path.join('data', 'test-gold', 'STS.input.' + file + '.txt'), 'r') as f:
        test_data += [sent.split('\t') for sent in f.readlines()]
    with open(os.path.join('data', 'test-gold', 'STS.gs.'+ file + '.txt'), 'r') as f:
        test_labels += [float(num) for num in f.readlines()]
train = list(zip(train_data, train_labels))
test = list(zip(test_data, test_labels))



## Validate

In [72]:
from scipy import stats
import sklearn
def evaluate(true_labels, predicted_labels):
    pearson, p_value = stats.pearsonr(true_labels, predicted_labels)
    return pearson, p_value
def cross_validate(data, labels, model, n_folds=5, seed=1):
    kf = sklearn.model_selection.KFold(n_splits=n_folds, random_state=seed)
    average_pearson = 0
    for train_index, test_index in kf.split(data):
        X_train, X_val = X[train_index], X[val_index]
        y_train, y_val = y[train_index], y[val_index]
        m = model.fit(data[X_train], labels[X_train])
        predicted_labels = model.predict(data[y_val])
        pearson, _ = evaluate(labels[y_val], predicted_labels)
        average_pearson += pearson
    return average_pearson/kf

## Evaluation

In [69]:
from datetime import datetime

def evaluate(true_labels, predicted_labels):
    print('NOTE: The official Perl scripts requires installing the Statistics::Basic module')
    print('curl -L http://cpanmin.us | perl - --sudo Statistics::Basic')
    print()
    assert len(true_labels) == len(predicted_labels)
    true_labels = list(map(str, true_labels))
    true_labels = list(map(lambda x: x + '\n', true_labels))
    predicted_labels = list(map(str, predicted_labels))
    predicted_labels = list(map(lambda x: x + '\t100\n', predicted_labels))
    print(predicted_labels)
    tmp = os.path.join('data', 'tmp')
    if not os.path.exists(tmp):
        os.makedirs(tmp)
    timestamp = 'output_{date:%Y-%m-%d_%H:%M:%S}.txt'.format(date=datetime.now())
    os.makedirs(os.path.join(tmp, timestamp))
    predicted_path = os.path.join(tmp, timestamp, 'predicted.txt')
    with open(predicted_path, 'w') as f:
        f.writelines(predicted_labels)
    true_path = os.path.join(tmp, timestamp, 'true.txt')
    with open(true_path, 'w') as f:
        f.writelines(true_labels)
    res_path = os.path.join(tmp, timestamp, 'res.txt')
    cmd = os.path.join('data', 'train', 'correlation.pl') + ' ' + true_path + ' ' + predicted_path
    print(cmd)
    res = os.popen(cmd).read()
    with open(res_path, 'w') as f:
        f.write(res)
    return res